In [ ]:

from __future__ import division,print_function


import theano
import keras
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense, Activation, TimeDistributed,RepeatVector
from keras.layers import LSTM
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from sklearn.cross_validation import train_test_split
from keras.optimizers import SGD,Adam
from  data_processing import data_process
from keras.layers import Dense,Input, Activation
import keras
import time
import numpy as np
np.random.seed(1337)  # for reproducibility
from keras.layers import Convolution1D, MaxPooling1D,Flatten

from  data_processing_fast import data_process


In [ ]:

# this function calculates Phoneme error rate (PER)
def per(r, h):
    #build the matrix
    d = np.zeros((len(r)+1)*(len(h)+1), dtype=np.uint8).reshape((len(r)+1, len(h)+1))
    for i in range(len(r)+1):
        for j in range(len(h)+1):
            if i == 0: d[0][j] = j
            elif j == 0: d[i][0] = i
    for i in range(1,len(r)+1):
        for j in range(1, len(h)+1):
            if r[i-1] == h[j-1]:
                d[i][j] = d[i-1][j-1]
            else:
                substitute = d[i-1][j-1] + 1
                insert = d[i][j-1] + 1
                delete = d[i-1][j] + 1
                d[i][j] = min(substitute, insert, delete)
    result = float(d[len(r)][len(h)]) / len(r) * 100
    return result

# in below, I made my callbacks which after every epoch it calculates PER 
class My_Callback(keras.callbacks.Callback):
    def __init__(self):
        super(keras.callbacks.Callback,self).__init__()
    def on_train_begin(self, logs={}):
        return
    
    def on_train_end(self, logs={}):
        return
    
    def on_epoch_begin(self,epoch, logs={}):
        return
    
    def on_epoch_end(self, epoch, logs={}):
        x_validation=self.validation_data[0]
        y_validation=self.validation_data[1]
        y_validation=y_validation.reshape(y_validation.shape[0],maxlen,len_out)
        pr=0
        er_word=0
        for i in range(x_validation.shape[0]):
            x_test = np.array([x_validation[i, :, :]])
            y_test = np.array([y_validation[i, :, :]])
            y_pred = model.predict(x_test, verbose=0)
            y_pred=y_pred.reshape(y_pred.shape[0],maxlen,len_out)

            xtest_text = decode_text_i(x_test[0])
            ytest_text = decode_text(y_test[0])
            ypred_text = decode_text(y_pred[0])
            if ypred_text not in ytest_text:
                er_word=er_word+1
            o,p = ytest_text.split(),ypred_text.split()
            pr=pr+per(o,p)
        
        PER=pr/x_validation.shape[0]
        WER=er_word/x_validation.shape[0]
        logs['val_PER']=PER
        logs['val_WER']=WER
        print(' ')
        print('val_PER: ', PER)
        print(' ')

       
        
        return    
    def on_batch_begin(self, batch, logs={}):
        return
    
    def on_batch_end(self, batch, logs={}):
        #self.losses.append(logs.get('loss'))
        return


In [ ]:
def decode_text_i(probas):
    text_seq = []
    for i in range(probas.shape[0]):
        idx = np.argmax(probas[i])
        text_seq.append(indices_char_i[idx])
    return "".join(text_seq).strip()

def decode_text(probas):
    text_seq = []
    for i in range(probas.shape[0]):
        idx = np.argmax(probas[i])
        text_seq.append(indices_char_o[idx])
        dec_text=" ".join(text_seq).strip()
        if indices_char_o[idx]=='/EP' :
            break
    return dec_text

In [ ]:
start_time =time.time()



path1='training_input.txt'
path2='training_output.txt'
path3='test_input.txt'
path4='test_output.txt'
path5='dev_input.txt' # validation data
path6='dev_output.txt'



X,Y, Xdev,Ydev,Xtest,Ytest,maxlen,len_in,len_out,indices_char_o=data_process_fast(path1,path2,path3,path4,path5,path6)


X=X.reshape(X.shape[0],X.shape[1],1).astype("float32")
Xtest=Xtest.reshape(Xtest.shape[0],Xtest.shape[1],1).astype("float32")
Xdev=Xdev.reshape(Xdev.shape[0],Xdev.shape[1],1).astype("float32")
Y=Y.reshape(Y.shape[0],-1).astype("float32")
Ydev=Ydev.reshape(Ydev.shape[0],-1).astype("float32")



In [ ]:
    
print ('Model')

inp = Input((X.shape[1],X.shape[2]))
cnv1 = Convolution1D(64, 3, border_mode='same')(inp)

r1 = Residual(64, 128, cnv1)
r2 = Residual(128, 256, r1)
r3 = Residual(256, 512, r2)

btc2 = BatchNormalization()(r3) 
act2 = Activation("relu")(btc2)
flat=Flatten()(act2)


out= Dense(len_out*maxlen,activation='softmax') (flat)
model = Model(input=inp, output=out)

model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=['accuracy'])
model.summary()

early_stopping = EarlyStopping(monitor='val_PER', patience=60, mode='min')
filepath="residual_cnn_dep3.hd5"
checkpoint = ModelCheckpoint(filepath, monitor='val_PER', save_best_only=True,save_weights_only=True, mode='min')
callbacks_list = [My_Callback(),early_stopping,checkpoint]
model.fit(X, Y, validation_data=(Xdev, Ydev), batch_size=128, nb_epoch=100, callbacks=callbacks_list)
model.load_weights("residual_cnn_dep3.hd5")




In [ ]:
er_word=0 #how many word is not correct
phonem=0 
for i in range(Xtest.shape[0]):  
    x_test = np.array([Xtest[i, :, :]])
    y_test = np.array([Ytest[i, :, :]])
    y_pred = model.predict(x_test, verbose=0)
    y_pred=y_pred.reshape(y_pred.shape[0],maxlen,len_out)
    ytest_text = decode_text(y_test[0])
    ypred_text = decode_text(y_pred[0])
    
    print("expected: [%s], got: [%s] " % 
    ( ytest_text, ypred_text))
         
    if ypred_text not in ytest_text:
        er_word=er_word+1
    o,p = ytest_text.split(),ypred_text.split()
    phonem=phonem+per(o,p)


print('how many words are wrong: ', er_word )
print('Phoneme error rate:', phonem/Xtest.shape[0])
print('Word error rate:',  er_word/Xtest.shape[0])

print("--- %s seconds ---" % (time.time() - start_time))

